## ENS ABI Research in the web3.py

In [1]:
import pandas as pd
import numpy as np
from ens import ENS
from web3 import Web3, HTTPProvider
from web3._utils.normalizers import validate_abi
from tqdm.notebook import tqdm
import json

from config import ETH_URL

tqdm.pandas()

w3 = Web3(HTTPProvider(ETH_URL))
ns = ENS.fromWeb3(w3)

### ABIs from [web3.py](https://github.com/ethereum/web3.py/blob/master/ens/abis.py)
- ENS
- DEED
- FIFS_REGISTRAR
- RESOLVER
- REVERSE_RESOLVER
- REVERSE_REGISTRAR

In [2]:
print(f'''
- ENS (ENSRegistryWithFallback)  {ns.ens.address}
- DEED
- FIFS_REGISTRAR
- RESOLVER (PublicResolver) example  {ns.resolver(name="1inch.eth").address}
- REVERSE_RESOLVER
- REVERSE_REGISTRAR (ReverseRegistrar)  {ns._reverse_registrar().address}

Other contracts
- Old ENS Token contract (BaseRegistrarImplementation)  0xfac7bea255a6990f749363002136af6556b31e04
- ENS Token contract (BaseRegistrarImplementation)  0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85
- ETHRegistrarController 1  0xf0ad5cad05e10572efceb849f6ff0c68f9700455
- ETHRegistrarController 2  0xb22c1c159d12461ea124b0deb4b5b93020e6ad16
- ETHRegistrarController 3  0x283af0b28c62c092c9727f1ee09c02ca627eb7f5
''')


- ENS (ENSRegistryWithFallback)  0x00000000000C2E074eC69A0dFb2997BA6C7d2e1e
- DEED
- FIFS_REGISTRAR
- RESOLVER (PublicResolver) example  0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41
- REVERSE_RESOLVER
- REVERSE_REGISTRAR (ReverseRegistrar)  0x084b1c3C81545d370f3634392De611CaaBFf8148

Other contracts
- Old ENS Token contract (BaseRegistrarImplementation)  0xfac7bea255a6990f749363002136af6556b31e04
- ENS Token contract (BaseRegistrarImplementation)  0x57f1887a8bf19b14fc0df6fd9b2acc9af147ea85
- ETHRegistrarController 1  0xf0ad5cad05e10572efceb849f6ff0c68f9700455
- ETHRegistrarController 2  0xb22c1c159d12461ea124b0deb4b5b93020e6ad16
- ETHRegistrarController 3  0x283af0b28c62c092c9727f1ee09c02ca627eb7f5



### Actual Resolver Contracts

In [3]:
ens_data_df = pd.read_csv('data/ens_data_row.csv', index_col=0)
resolver_contracts_df = ens_data_df.groupby('resolver_address').name.agg(name_cnt=np.count_nonzero).reset_index().sort_values('name_cnt', ascending=False).reset_index()
resolver_contracts_list = resolver_contracts_df.resolver_address.loc[:6].to_list()
resolver_contracts_df

,index,resolver_address,name_cnt
0,80,0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41,1881273
1,36,0x226159d592E2b063810a10Ebf6dcbADA94Ed68b8,2571
2,217,0xD3ddcCDD3b25A8a7423B5bEe360a42146eb4Baf3,1495
3,31,0x1da022710dF5002339274AaDEe8D58218e9D6AB5,1345
4,175,0x9C4e9CCE4780062942a7fe34FA2Fa7316c872956,489
...,...,...,...
289,119,0x721B682B80F427c52057BCC4B68E85B13C5e4b48,1
290,120,0x72CBdEaAdddD14Ec95b92995933CeC69566650f0,1
291,121,0x74312363e45DCaBA76c59ec49a7Aa8A65a67EeD3,1
292,122,0x74740c72e7b667344084772D2fdBB4646099c705,1


### Compare Resolver Contracts ABIs

In [4]:
with open('data/abi/resolver_web3py_abi.json', 'r') as resolver_web3py_file:
    resolver_web3py_json = json.load(resolver_web3py_file)
    resolver_web3py_abi_list = [item['name'] if 'name' in item.keys() else None for item in resolver_web3py_json]

resolver_abi_json = {}
resolver_abi_list = {}
for resolver_contract in resolver_contracts_list:
    with open(f'data/abi/resolver_{resolver_contract}_abi.json', 'r') as resolver_file:
        resolver_abi_json[resolver_contract] = json.load(resolver_file)
        resolver_abi_list[resolver_contract] = [item['name'] if 'name' in item.keys() else None for item in resolver_abi_json[resolver_contract]]

In [5]:
not_in_main_resolver_contract_abi_dict = {}
main_resolver_contract = resolver_contracts_list[0]
for resolver_contract in resolver_contracts_list:
    if resolver_contract != main_resolver_contract:
        print(
            f'''\n{resolver_contract}
        Common abi methods \n\t\t{
            ', '.join(abi_item for abi_item in set(
                item for item in resolver_abi_list[main_resolver_contract] if item is not None and item in resolver_abi_list[resolver_contract]))}
        Not in resolver contract abi \n\t\t{
            ', '.join(abi_item for abi_item in set(resolver_abi_list[main_resolver_contract]) - set(resolver_abi_list[resolver_contract]))}
        Not in main resolver contract abi \n\t\t{
            ', '.join(abi_item for abi_item in set(resolver_abi_list[resolver_contract]) - set(resolver_abi_list[main_resolver_contract]))}''')
        if set(resolver_abi_list[resolver_contract]) - set(resolver_abi_list[main_resolver_contract]):
            not_in_main_resolver_contract_abi_dict[resolver_contract] = set(resolver_abi_list[resolver_contract]) - set(resolver_abi_list[main_resolver_contract])


0x226159d592E2b063810a10Ebf6dcbADA94Ed68b8
        Common abi methods 
		AddrChanged, setText, setAuthorisation, setABI, setName, setAddr, addr, NameChanged, authorisations, AuthorisationChanged, setContenthash, contenthash, setInterface, PubkeyChanged, setPubkey, supportsInterface, pubkey, ABIChanged, ABI, interfaceImplementer, text, AddressChanged, TextChanged, ContenthashChanged, name, InterfaceChanged
        Not in resolver contract abi 
		setDNSRecords, clearDNSZone, DNSRecordDeleted, hasDNSRecords, dnsRecord, DNSRecordChanged, multicall, DNSZoneCleared
        Not in main resolver contract abi 
		

0xD3ddcCDD3b25A8a7423B5bEe360a42146eb4Baf3
        Common abi methods 
		AddrChanged, setText, setABI, setName, setAddr, addr, NameChanged, setContenthash, contenthash, PubkeyChanged, setPubkey, supportsInterface, pubkey, ABIChanged, ABI, text, TextChanged, ContenthashChanged, name
        Not in resolver contract abi 
		interfaceImplementer, setDNSRecords, AddressChanged, setInterfa

In [6]:
not_in_main_resolver_contract_abi_reverse_dict = {}
for element, k in [[element, k] for k,v in not_in_main_resolver_contract_abi_dict.items() for element in v]:
    if element in not_in_main_resolver_contract_abi_reverse_dict.keys():
        not_in_main_resolver_contract_abi_reverse_dict[element].append(k)
    else:
        not_in_main_resolver_contract_abi_reverse_dict[element] = [k]
print(json.dumps(not_in_main_resolver_contract_abi_reverse_dict, indent=4, sort_keys=True))

{
    "ContentChanged": [
        "0x1da022710dF5002339274AaDEe8D58218e9D6AB5",
        "0x5FfC014343cd971B7eb70732021E26C35B744cc4"
    ],
    "DNSZonehashChanged": [
        "0xB37671329ABE589109b0bDD1312cc6ACcF106259"
    ],
    "OwnershipTransferred": [
        "0x9C4e9CCE4780062942a7fe34FA2Fa7316c872956"
    ],
    "StealthKeyChanged": [
        "0xB37671329ABE589109b0bDD1312cc6ACcF106259"
    ],
    "addTokenId": [
        "0x9C4e9CCE4780062942a7fe34FA2Fa7316c872956"
    ],
    "baseURI": [
        "0x9C4e9CCE4780062942a7fe34FA2Fa7316c872956"
    ],
    "content": [
        "0x1da022710dF5002339274AaDEe8D58218e9D6AB5",
        "0x5FfC014343cd971B7eb70732021E26C35B744cc4"
    ],
    "fallbackResolver": [
        "0xB37671329ABE589109b0bDD1312cc6ACcF106259"
    ],
    "getTokenId": [
        "0x9C4e9CCE4780062942a7fe34FA2Fa7316c872956"
    ],
    "isOwner": [
        "0x9C4e9CCE4780062942a7fe34FA2Fa7316c872956"
    ],
    "openSeaVersion": [
        "0x9C4e9CCE4780062942a7fe34FA2Fa

### Compare Main Resolver and web3py ABI methods

In [7]:
main_resolver_contract = '0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41'

elements_in_resolver_json = {resolver_contract: {element['name']: element for element in resolver_abi_json[resolver_contract] if 'name' in element.keys()} for resolver_contract in resolver_contracts_list }

In [8]:
def compare_elements(element1: dict, element2: dict, element_name: str, display_detailed_data: bool = False,
                     ignore_field: tuple = ('indexed', 'internalType', 'name', 'constant'),
                     ignore_key: tuple = ('stateMutability', 'payable', 'constant')) -> bool:
    _element_match = True
    for _key in element1.keys():
        element1_item_json = element1[_key]
        element2_item_json = element2[_key] if _key in element2.keys() else {}
        if _key in ('inputs', 'outputs'):
            element1_item_json = \
                [{key: item[key] for key in item.keys() if key not in ignore_field} for item in element1_item_json]
            element2_item_json = \
                [{key: item[key] for key in item.keys() if key not in ignore_field} for item in element2_item_json]
        if _key not in ignore_key and element1_item_json != element2_item_json:
            if _element_match:
                print(f'\t{element_name}')
            print(f'\t\t{_key}')
            if display_detailed_data:
                print('\t\t\t- main resolver')
                print(json.dumps(element1[_key], indent=4, sort_keys=True))
                print('\t\t\t- other resolver')
                print(json.dumps(element2[_key] if _key in element2.keys() else {}, indent=4, sort_keys=True))
            _element_match = False
    return _element_match


for resolver_contract in resolver_contracts_list:
    if resolver_contract != main_resolver_contract:
        print(f'\n\n{resolver_contract}')
        common_methods = set(element for element in resolver_abi_list[resolver_contract] if element is not None and element in resolver_abi_list[main_resolver_contract])
        elements_match_list = []
        for common_method in common_methods:
           elements_match_list.append(compare_elements(
                element1=elements_in_resolver_json[main_resolver_contract][common_method],
                element2=elements_in_resolver_json[resolver_contract][common_method],
                element_name=f"{elements_in_resolver_json[main_resolver_contract][common_method]['type']}: {common_method}"))
        if elements_match_list == [True] * len(elements_match_list):
            print('\tmatched')



0x226159d592E2b063810a10Ebf6dcbADA94Ed68b8
	matched


0xD3ddcCDD3b25A8a7423B5bEe360a42146eb4Baf3
	function: addr
		inputs
		outputs


0x1da022710dF5002339274AaDEe8D58218e9D6AB5
	function: addr
		inputs
		outputs


0x9C4e9CCE4780062942a7fe34FA2Fa7316c872956
	function: addr
		inputs
		outputs


0x5FfC014343cd971B7eb70732021E26C35B744cc4
	function: addr
		inputs
		outputs


0xB37671329ABE589109b0bDD1312cc6ACcF106259
	matched


#### Not matched 4 from top 7 resolvers (0.2% of names)
- function
    - addr (different inputs field and outputs type)

### Final ABI for web3.py



In [9]:
print('All ABI elements from Main Resolver:')
print('- functions\n\t-','''\n\t- '''.join(
    element['name'] for element in resolver_abi_json[main_resolver_contract] if element['type'] == 'function'))
print('- events\n\t-','''\n\t- '''.join(
    element['name'] for element in resolver_abi_json[main_resolver_contract] if element['type'] == 'event'))
print('- constructor')

print('\nABI elements from other Resolvers:')
print('- functions\n\t-', '''\n\t- '''.join(
    f'{element_name}: {", ".join(contract_list)}'
    for element_name, contract_list in not_in_main_resolver_contract_abi_reverse_dict.items()
    if [element['type'] for element in resolver_abi_json[contract_list[0]] if 'name' in element.keys() and element['name']==element_name][0] == 'function'))
print('- events\n\t-', '''\n\t- '''.join(
    f'{element_name}: {", ".join(contract_list)}'
    for element_name, contract_list in not_in_main_resolver_contract_abi_reverse_dict.items()
    if [element['type'] for element in resolver_abi_json[contract_list[0]] if 'name' in element.keys() and element['name']==element_name][0] == 'event'))

All ABI elements from Main Resolver:
- functions
	- ABI
	- addr
	- addr
	- authorisations
	- clearDNSZone
	- contenthash
	- dnsRecord
	- hasDNSRecords
	- interfaceImplementer
	- multicall
	- name
	- pubkey
	- setABI
	- setAddr
	- setAddr
	- setAuthorisation
	- setContenthash
	- setDNSRecords
	- setInterface
	- setName
	- setPubkey
	- setText
	- supportsInterface
	- text
- events
	- ABIChanged
	- AddrChanged
	- AddressChanged
	- AuthorisationChanged
	- ContenthashChanged
	- DNSRecordChanged
	- DNSRecordDeleted
	- DNSZoneCleared
	- InterfaceChanged
	- NameChanged
	- PubkeyChanged
	- TextChanged
- constructor

ABI elements from other Resolvers:
- functions
	- setContent: 0x1da022710dF5002339274AaDEe8D58218e9D6AB5, 0x5FfC014343cd971B7eb70732021E26C35B744cc4
	- content: 0x1da022710dF5002339274AaDEe8D58218e9D6AB5, 0x5FfC014343cd971B7eb70732021E26C35B744cc4
	- transferOwnership: 0x9C4e9CCE4780062942a7fe34FA2Fa7316c872956
	- setBaseURI: 0x9C4e9CCE4780062942a7fe34FA2Fa7316c872956
	- renounceOwn

In [10]:
for element_name, contract_list in not_in_main_resolver_contract_abi_reverse_dict.items():
    if len(contract_list) > 1:
        print(f'element: {element_name} \ncontracts: {", ".join(contract_list)}')
        element_matched = compare_elements(element1=[element for element in resolver_abi_json[contract_list[0]] if 'name' in element.keys() and element['name']==element_name][0],
                         element2=[element for element in resolver_abi_json[contract_list[1]] if 'name' in element.keys() and element['name']==element_name][0],
                         element_name=element_name)
        if element_matched:
            print('\tmatched')

element: ContentChanged 
contracts: 0x1da022710dF5002339274AaDEe8D58218e9D6AB5, 0x5FfC014343cd971B7eb70732021E26C35B744cc4
	matched
element: setContent 
contracts: 0x1da022710dF5002339274AaDEe8D58218e9D6AB5, 0x5FfC014343cd971B7eb70732021E26C35B744cc4
	matched
element: content 
contracts: 0x1da022710dF5002339274AaDEe8D58218e9D6AB5, 0x5FfC014343cd971B7eb70732021E26C35B744cc4
	matched


### Final ABI for Resolver Contract [here](data/abi/resolver_final_abi.json)

In [11]:
resolver_final_abi_json = resolver_abi_json[main_resolver_contract].copy()
for element_name, contract_list in not_in_main_resolver_contract_abi_reverse_dict.items():
    resolver_final_abi_json.append([element for element in resolver_abi_json[contract_list[0]] if 'name' in element.keys() and element['name']==element_name][0])

In [12]:
# remove `internalType` field to fit the format web3.py
resolver_final_abi_json = \
    [{
        key:
            [{k: v for k, v in item.items() if k != 'internalType'}
             for item in value]
            if key in ('inputs', 'outputs')
            else value
        for key, value in element.items()
    }
    for element in resolver_final_abi_json]

validate_abi(resolver_final_abi_json)

In [13]:
# format `inputs` and `outputs` to fit the format web3.py
resolver_final_abi_str = json.dumps(resolver_final_abi_json, indent=2, sort_keys=True)\
            .replace('"inputs": [\n      {\n        ', '"inputs": [\n      {')\
            .replace('{\n        "indexed"', '{"indexed"')\
            .replace(',\n        "name"', ', "name"')\
            .replace('{\n        "name"', '{"name"')\
            .replace('",\n        "type"', '", "type"')\
            .replace('\n      },\n ', '},\n ')\
            .replace('"\n      }\n    ]', '"}\n    ]')
with open('data/abi/resolver_final_abi.json', 'w') as resolver_final_file:
    resolver_final_file.write(resolver_final_abi_str)
print(resolver_final_abi_str)

[
  {
    "inputs": [
      {"name": "_ens", "type": "address"}
    ],
    "payable": false,
    "stateMutability": "nonpayable",
    "type": "constructor"
  },
  {
    "anonymous": false,
    "inputs": [
      {"indexed": true, "name": "node", "type": "bytes32"},
      {"indexed": true, "name": "contentType", "type": "uint256"}
    ],
    "name": "ABIChanged",
    "type": "event"
  },
  {
    "anonymous": false,
    "inputs": [
      {"indexed": true, "name": "node", "type": "bytes32"},
      {"indexed": false, "name": "a", "type": "address"}
    ],
    "name": "AddrChanged",
    "type": "event"
  },
  {
    "anonymous": false,
    "inputs": [
      {"indexed": true, "name": "node", "type": "bytes32"},
      {"indexed": false, "name": "coinType", "type": "uint256"},
      {"indexed": false, "name": "newAddress", "type": "bytes"}
    ],
    "name": "AddressChanged",
    "type": "event"
  },
  {
    "anonymous": false,
    "inputs": [
      {"indexed": true, "name": "node", "type": "byt

### Check `content` and `contenthash` functions

In [14]:
for resolver_contract in resolver_contracts_list:
    print(f'Resolver {resolver_contract}')
    try:
        any_name = ens_data_df[
                       (~ens_data_df.contenthash.isna()) &
                       (ens_data_df.contenthash != '0000000000000000000000000000000000000000000000000000000000000000') &
                       (ens_data_df.resolver_address == resolver_contract)].loc[:,'name'].to_list()[0]
        print(f'\tName: {any_name}\n\tOwner {ns.owner(any_name)}')
        print(f'\tResolver address: {ns.resolver(name=any_name).address}')
        try:
            print(f'\tContent {ns.resolver(name=any_name).functions.contenthash(ns.namehash(name=any_name)).call().hex()}')
        except ValueError:
            print(f'\tContent {ns.resolver(name=any_name).functions.content(ns.namehash(name=any_name)).call().hex()}')
    except IndexError:
        print('- no content items')

Resolver 0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41
	Name: 0-1337.eth
	Owner 0x8F82De72659D5a1670d3Db7C440A8E0a6E925E02
	Resolver address: 0x4976fb03C32e5B8cfe2b6cCB31c09Ba78EBaBa41
	Content e30101701220d6f0e0d157170f52052e3f81bdafafb2e3d484a6d67519363dcaab92e53611ca
Resolver 0x226159d592E2b063810a10Ebf6dcbADA94Ed68b8
	Name: 1website.eth
	Owner 0x3E204F427Bd69e2ee7402a71fb1CbdD616B40200
	Resolver address: 0x226159d592E2b063810a10Ebf6dcbADA94Ed68b8
	Content e30101701220d2b4f9aa5e03b703ad961bd57f28fd8f130eecc69e35ccd078fe78f6f2ebf776
Resolver 0xD3ddcCDD3b25A8a7423B5bEe360a42146eb4Baf3
	Name: 0daytoday.eth
	Owner 0xFdC658d9f188EC2334a35109EA28C195D6880bE9
	Resolver address: 0xD3ddcCDD3b25A8a7423B5bEe360a42146eb4Baf3
	Content bc036d76666a66756764776763357577686f
Resolver 0x1da022710dF5002339274AaDEe8D58218e9D6AB5
	Name: 3dhdwallpapers.eth
	Owner 0xCEd576598F193fCcB4c263dF67658Bed695e52b7
	Resolver address: 0x1da022710dF5002339274AaDEe8D58218e9D6AB5
	Content 9af7f4a3d9548f37b0c6e391fa8b1e23